# 1.3 Crash Course by Example
## 1.3.1 Bubble Sort

In [1]:
function BubbleSort!(a::Array)
    for i in collect(1:length(a)-1)
        for j in collect(1:length(a)-i)
            if a[j] > a[j+1]
                a[j], a[j+1] = a[j+1], a[j]
            end
        end
    end 
    return a   
end

data = floor.(rand(10)*10)
BubbleSort!(data)

10-element Vector{Float64}:
 0.0
 0.0
 1.0
 3.0
 5.0
 6.0
 8.0
 8.0
 9.0
 9.0

## 1.3.2 Roots of a Polynomia
$$
f(x) = a_{n}x^{n} + a_{n-1}x^{n-1} + \cdots + a_{1}x + a_{0} = \sum_{i=0}^{n}a_{i}x^{i}
$$

- This example is find all $x$ values that solve the equaiton $f(x)=-10x^{2}+3x+1 = 0$

### Knowledges about Julia
- `...` indicates that this function will accept comma separated parameters with an unspecified number of parameters.

In [2]:
using Roots

function polynomiaGenerator(a...)
    n = length(a) - 1
    poly = function(x)
        return sum([a[i+1]*x^i for i in 0:n])
    end
    return poly
end

@time begin
    polynomia = polynomiaGenerator(1, 3, -10)
    zeroVals = find_zeros(polynomia, -10, 10)    
end
println("Zeros of the function f(x): ", zeroVals)

  1.677552 seconds (3.83 M allocations: 222.090 MiB, 11.01% gc time, 99.98% compilation time)
Zeros of the function f(x): [-0.19999999999999998, 0.5]


In [3]:
function polynomiaGenerator2(a...)
    f(x) = sum([a[i+1]*x^i for i in 0:length(a)-1])
    return f
end

@time begin
    polynomia2 = polynomiaGenerator2(1, 3, -10)
    zeroVals = find_zeros(polynomia2, -10, 10)    
end

  0.200509 seconds (610.16 k allocations: 35.747 MiB, 5.27% gc time, 99.87% compilation time)


2-element Vector{Float64}:
 -0.19999999999999998
  0.5

## 1.3.3 Markov Chain
- ***Transition probability matrix*** $\mathbf{P}$: $P_{i,j}$ describes the probability of transitioning to $j$ state from current $i$ state
  - i.e. The $i$ state is Rain today and the $j$ state is Fine next day, the $P_{i,j}$ is the probability of the next day is Fine when today is Rain.
  - Thus $\sum_{j}{P_{i,j}} = 1$
- ***Stationary distribution*** $\boldsymbol{\pi}$

In [4]:
using LinearAlgebra, StatsBase


P = [0.5 0.4 0.1;
    0.3 0.2 0.5;
    0.5 0.3 0.2];

# First way
piProb1 = (P^100)[1,:]

# Second way
A = vcat((P'-I)[1:2, :], ones(3)')
b = [0 0 1]'
piProb2 = A\b

# Third way
eigVecs = eigvecs(copy(P'))
highestVec = eigVecs[:, findmax(abs.(eigvals(P)))[2]]
piProb3 = Array{Float64}(highestVec)/norm(highestVec,1)

# Fourth way
numInState = zeros(Int,3)
state = 1
N = 10^7
for _ in 1:N
    numInState[state] += 1
    global state = sample(1:3, weights(P[state,:]))
end
piProb4 = numInState/N

display([piProb1 piProb2 piProb3 piProb4])

3×4 Matrix{Float64}:
 0.4375  0.4375  0.4375  0.437744
 0.3125  0.3125  0.3125  0.312404
 0.25    0.25    0.25    0.249852

## 1.3.4 Web Interfacing, JSON, and String Processing

In [5]:
using HTTP, JSON

data = HTTP.request("GET",
"https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt")
shakespeare = String(data.body)
shakespeareWords = split(shakespeare)

# jsonWords = HTTP.request("GET",
# "https://raw.githubusercontent.com/"*
# "h-Klok/StatsWithJuliaBook/master/data/jsonCode.json")
jsonWords = """
{
  "words": [ "heaven","hell","man","woman","boy","girl","king","queen",
	"prince","sir","love","hate","knife","english","england","god"],
  "numToShow": 5
}
""";
parseJsonDict = JSON.parse(jsonWords)

keywords = Array{String}(parseJsonDict["words"])
numberToShow = parseJsonDict["numToShow"]
wordCount = Dict([(x, count(w -> lowercase(w) == lowercase(x), shakespeareWords)) for x in keywords])

sortedWordCount = sort(collect(wordCount), by=last, rev=true)
display(sortedWordCount[1:numberToShow])

5-element Vector{Pair{String, Int64}}:
 "king" => 1698
 "love" => 1279
  "man" => 1033
  "sir" => 721
  "god" => 555